# NanoEventsFactory example

Load a single file from x-cache (using redirector_ucsd) or the grid (using redirector_fnal), and play with the events.


In [13]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import Weights, PackedSelection

from Tools.config_helpers import redirector_fnal, redirector_ucsd
from Tools.nano_mapping import make_fileset
from Tools.helpers import get_samples

import awkward as ak

samples = get_samples()

fileset = make_fileset(['top'], samples, redirector=redirector_ucsd, small=True)

# load a subset of events
n_max = 50000000
events = NanoEventsFactory.from_root(
    fileset[list(fileset.keys())[0]][0],
    schemaclass = NanoAODSchema,
    entry_stop = n_max).events()



In [14]:
events.MET.pt

<Array [51.5, 26.5, 42, ... 67.6, 34.6, 114] type='1140000 * float32[parameters=...'>

In [15]:
import time

from Tools.objects import Collections

start_time = time.time()
electrons = Collections(events, 'Electron', 'tight', verbose=1).get()

delta_time = time.time()-start_time

print ("\nTook %s seconds"%delta_time)


## Electron selection for WP tight ##
 - pt >= 15
 - abs(etaSC) <= 2.5
 - convVeto == 1
 - lostHits == 0
 - tightCharge == 2
 - abs(dxy) <= 0.05
 - abs(dz) <= 0.1
 - abs(sip3d) <= 4.0
 - tight electron MVA ID
 - custom multi isolation
 - trigger safe
 - custom ID and multi-isolation
Found 418141 objects passing the selection

Took 61.637818574905396 seconds


In [16]:
electrons.eta

<Array [[], [], [], ... [-0.892], []] type='1140000 * var * float32[parameters={...'>

You can do anything that you're usually doing inside the processor here as well, so this is very useful for debugging

In [21]:
matched_electrons = electrons[electrons.genPartIdx>=0]
sum(ak.num(matched_electrons, axis=1))

415651

In [25]:
test_ar = ak.to_numpy(ak.flatten(abs(matched_electrons.eta)))
test_ar[test_ar>2.5]

AttributeError: no field named 'etaSc'

(https://github.com/scikit-hep/awkward-1.0/blob/1.2.2/src/awkward/highlevel.py#L1094)